# Lightweight python components - share component files

Lightweight python components do not require you to build a new container image for every code change.
They're intended to use for fast iteration in notebook environment.

#### Building a lightweight python component
To build a component just define a stand-alone python function and then call kfp.components.func_to_container_op(func) to convert it to a component that can be used in a pipeline.

There are several requirements for the function:
* The function should be stand-alone. It should not use any code declared outside of the function definition. Any imports should be added inside the main function. Any helper functions should also be defined inside the main function.
* The function can only import packages that are available in the base image. If you need to import a package that's not available you can try to find a container image that already includes the required packages. (As a workaround you can use the module subprocess to run pip install for the required package.)
* If the function operates on numbers, the parameters need to have type hints. Supported types are ```[int, float, bool]```. Everything else is passed as string.
* To build a component with multiple output values, use the typing.NamedTuple type hint syntax: ```NamedTuple('MyFunctionOutputs', [('output_name_1', type), ('output_name_2', float)])```

In [ ]:
#Install the SDK
!pip3 install https://storage.googleapis.com/ml-pipeline/release/0.1.1/kfp.tar.gz --upgrade


In [2]:
import kfp.components as comp

Simple function that just add two numbers:

In [3]:
def add(a: float, b: float) -> float:
   '''Calculates sum of two arguments'''
   return a + b

Convert the function to a pipeline operation. This time specify the `output_component_file` argument to create a component file that can be shared

In [4]:
add_op = comp.func_to_container_op(add, output_component_file='add.component')

Advanced function.
Demonstrates how to use imports, helper functions and produce multiple outputs.

In [5]:
#Advanced function
#Demonstrates imports, helper functions and multiple outputs
from typing import NamedTuple
def my_divmod(dividend: float, divisor:float) -> NamedTuple('MyDivmodOutput', [('quotient', float), ('remainder', float)]):
    '''Divides two numbers and calculate  the quotient and remainder'''
    #Imports inside a component function:
    import numpy as np

    #This function demonstrates how to use nested functions inside a component function:
    def divmod_helper(dividend, divisor):
        return np.divmod(dividend, divisor)

    (quotient, remainder) = divmod_helper(dividend, divisor)

    from collections import namedtuple
    divmod_output = namedtuple('MyDivmodOutput', ['quotient', 'remainder'])
    return divmod_output(quotient, remainder)

#### Convert the function to a pipeline operation

You can specify an alternative base container image (the image needs to have Python 3.5+ installed). This time specify the `output_component_file` argument to create a component file that can be shared

In [7]:
my_divmod_op = comp.func_to_container_op(my_divmod, base_image='tensorflow/tensorflow:1.11.0-py3', output_component_file='divmod.component')

#### Now continue to the Tutorial 2c - Lightweight Python components - sharing - consume component file